In [ ]:
# Ant Colony Simulation using ipycanvas
# Run this in a Jupyter Notebook
import random
import math
import numpy as np
import asyncio
import time
from ipycanvas import Canvas, hold_canvas
from IPython.display import display

from scipy.ndimage import gaussian_filter



def linear_probs(values, beta=0.05):
    total = sum(values) + beta * len(values)
    return [(v + beta) / total for v in values]

def softmax(values, beta=1.0):
    exps = [math.exp(beta * v) for v in values]
    s = sum(exps)
    return [e / s for e in exps]

def sense(p):
    return math.sqrt(p)


class Simulation(object):
    def __init__(self, shape, n_ants, pheromone_decay, pheromone_strength):
        self.width, self.height = shape
        self.n_ants = n_ants
        self.pheromone_decay = pheromone_decay
        self.pheromone_strength = pheromone_strength


        # ants  data as array
        self.ants = np.zeros([n_ants, 4], dtype=np.float32)  # x, y, angle, has_food

        # self.ants = [Ant(random.randint(0, self.width-1), random.randint(0, self.height-1)) for _ in range(self.n_ants)]

        # pheromone layer
        # channel 0: route to home pheromone
        # channel 1: route to food pheromone
        self.pheromone = np.zeros((self.width, self.height, 2))
        self.smooth_pheromone = np.zeros((self.width, self.height, 2))

        # is_home_map and is_food_map
        self.is_home_map = np.zeros((self.width, self.height), dtype=np.uint8)
        self.is_food_map = np.zeros((self.width, self.height), dtype=np.uint8)


        # for visualization
        self.canvas = Canvas(width=self.width, height=self.height)
        self.draw_array = np.zeros((self.width, self.height, 4), dtype=np.uint8)

        # display the canvas twice as big as the pixel size
        self.canvas.layout.width = f'{self.width * 2}px'
        self.canvas.layout.height = f'{self.height * 2}px'  
        display(self.canvas)

    def start(self):
        self.available_food = np.sum(self.is_food_map)
        self.returned_food = 0
    
    def step(self):
        # Decay pheromone
        self.pheromone *= self.pheromone_decay

        # # Soft clipping using tanh to prevent overflow while allowing gradual saturation
        # self.pheromone = 20 * np.tanh(self.pheromone / 20)

        # Smooth pheromone for better ant sensing
        for i in range(2):
            self.smooth_pheromone[:, :, i] = gaussian_filter(self.pheromone[:, :, i], sigma=5)
        
        self.smooth_pheromone += self.pheromone 

        # Move ants
        for i in range(self.n_ants):
            self.step_ant(i)


            
    def step_ant(self, ant_index):
        ant_data = self.ants[ant_index]
        x,y,angle,has_food = ant_data
        has_food = has_food > 0.5

        rx = int(x + 0.5)
        ry = int(y + 0.5)
        # wrap around
        rx %= self.width
        ry %= self.height
        old_rx, old_ry = rx, ry

        if has_food:
            # Check for home
            if self.is_home_map[rx, ry] == 1:
                ant_data[3] = 0  # drop food
                has_food = False 
                self.returned_food += 1 
                # turn around
                angle += math.pi

        else:
            # Check for food
            food_amount = self.is_food_map[rx, ry]
            if food_amount >= 1:
                ant_data[3] = 1  # pick up food
                has_food = True
                # reduce food amount
                self.is_food_map[rx, ry] = max(0, food_amount - 1)

                # turn around
                angle += math.pi
              
        int_x = int(x)
        int_y = int(y)

        # get the pheromone values in front and to the sides
        sensor_distance = 3.0
        sensor_angle_degrees = 30
        sensor_angle = math.radians(sensor_angle_degrees)

        front_x = int((x + sensor_distance * math.cos(angle))) % self.width
        front_y = int((y + sensor_distance * math.sin(angle))) % self.height

        left_x = int((x + sensor_distance * math.cos(angle + sensor_angle))) % self.width
        left_y = int((y + sensor_distance * math.sin(angle + sensor_angle))) % self.height
        right_x = int((x + sensor_distance * math.cos(angle - sensor_angle))) % self.width
        right_y = int((y + sensor_distance * math.sin(angle - sensor_angle))) % self.height

        pheromone_type = 0 if has_food else 1
        front_pheromone = self.smooth_pheromone[front_x, front_y, pheromone_type]
        left_pheromone = self.smooth_pheromone[left_x, left_y, pheromone_type]
        right_pheromone = self.smooth_pheromone[right_x, right_y, pheromone_type]  
        # max_pheromone = max(front_pheromone, left_pheromone, right_pheromone)
    

        beta = 3.0
        values = [
            sense(front_pheromone) + 0.85 ,
            sense(left_pheromone),
            sense(right_pheromone),
        ]
        probs = softmax(values, beta=beta)
        # print("phero", [front_pheromone, left_pheromone, right_pheromone])
        # print("vals", values)
        # print("probs", probs)

        choice = random.choices(['front', 'left', 'right'], weights=probs)[0]
        if choice == 'left':
            angle += sensor_angle
        elif choice == 'right':
            angle -= sensor_angle
        else:
            pass  # go straight


   
    

        ant_data[2] = angle
        # Move forward
        x += math.cos(angle)
        y += math.sin(angle)
        # Wrap edges
        x %= self.width
        y %= self.height
        ant_data[0] = x
        ant_data[1] = y
        
        rx = int(x + 0.5)
        ry = int(y + 0.5)
        rx %= self.width
        ry %= self.height

        # Deposit pheromone
        deposit_amount = self.pheromone_strength
        self.pheromone[int(old_rx), int(old_ry), int(has_food)] += deposit_amount 
    

    def draw(self):
        with hold_canvas(self.canvas):
            # Draw pheromone
            pheromone_home = self.smooth_pheromone[:, :, 0]
            pheromone_food = self.smooth_pheromone[:, :, 1]

            # Normalize for visualization
            max_pheromone = np.max(self.pheromone)
            if max_pheromone > 0:
                pheromone_home = (pheromone_home / max_pheromone * 255).astype(np.uint8)
                pheromone_food = (pheromone_food / max_pheromone * 255).astype(np.uint8)

            self.draw_array[:, :, 1] = pheromone_food  # green channel
            self.draw_array[:, :, 2] = pheromone_home  # blue channel
            self.draw_array[:, :, 0] = 0    # Red channel
            self.draw_array[:, :, 3] = 255  # Alpha channel


            # make the home blue and food green
            self.draw_array[self.is_home_map == 1] = [0, 0, 255, 255]
            self.draw_array[self.is_food_map == 1] = [0, 255, 0, 255]

            # we need to flip x and y for drawing
            self.draw_array = np.transpose(self.draw_array, (1, 0, 2))
            self.canvas.put_image_data(self.draw_array, 0, 0)


            # Draw ants
            with_food_ants = np.where(self.ants[:, 3] > 0.5)[0]
            without_food_ants = np.where(self.ants[:, 3] <= 0.5)[0]
            self.canvas.fill_style = 'blue'
            self.canvas.fill_circles(self.ants[without_food_ants, 0], self.ants[without_food_ants, 1], 2)
            self.canvas.fill_style = 'green'
            self.canvas.fill_circles(self.ants[with_food_ants, 0], self.ants[with_food_ants, 1], 2)

            # print available food and returned food
            self.canvas.fill_style = 'red'
            self.canvas.font = '16px Arial'
            self.canvas.fill_text(f'Available food: {self.available_food}', 10, 20)
            self.canvas.fill_text(f'Returned food: {self.returned_food}', 10, 40)





# --------------------
# World setup
# --------------------
# --------------------
# Parameters
# --------------------
WIDTH, HEIGHT = 300, 300
N_ANTS = 1000
PHEROMONE_DECAY = 0.999
PHEROMONE_STRENGTH = 1
STEP_DELAY = 0.01

sim = Simulation((WIDTH, HEIGHT), N_ANTS, PHEROMONE_DECAY, PHEROMONE_STRENGTH)


# make a 30x30 square for nest
nx, ny = sim.width // 2, sim.height // 2
sim.is_home_map[nx-15:nx+15, ny-15:ny+15] = 1

# start all ants at the nest
for ant_index in range(sim.n_ants):
    ant = sim.ants[ant_index]
    ant[0] = nx + random.uniform(-2, 2)
    ant[1] = ny + random.uniform(-2, 2)
    # random angle
    ant[2] = random.uniform(0, 2 * math.pi)
    ant[3] = 0  # no food  


# make a 30x30 square for food
fx, fy = 260, 270
sim.is_food_map[fx-15:fx+15, fy-15:fy+15] = 1

tstart = time.time()
sim.start()
i = 0
while time.time() - tstart < 200:  # run for 10 seconds
    sim.step()
    if i % 4 == 0:
        sim.draw()
    # time.sleep(STEP_DELAY)
    i += 1
    # if i > 10:
    #     break





Canvas(height=300, layout=Layout(height='600px', width='600px'), width=300)